In [79]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd

In [81]:
train3 = train

In [80]:
train = pd.read_csv("C:/dataset/dacon_SDSS/train.csv", index_col=0)
test = pd.read_csv("C:/dataset/dacon_SDSS/test.csv", index_col=0)
sample_submission = pd.read_csv("C:/dataset/dacon_SDSS/sample_submission.csv", index_col=0)

type : 천체의 분류(예측 변수)

psfMag : Point spread function magnitudes 빛의 밝기

fiberMag : 광섬유를 통과하는 빛의 밝기

petroMag : 빛의 밝기를 비교하기 위한 수치

medelMag : 천체 중심으로부터 특정 거리의 밝기

fiberID : 관측에 사용된 광섬유의 구분자

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
sample_submission.head()

In [ ]:
train.shape
test.shape
sample_submission.shape

In [ ]:
train.info()

In [ ]:
train['type'].unique()

In [ ]:
train['type'].value_counts()

In [ ]:
train['type'].nunique()

In [ ]:
train['fiberID'].nunique()

## 데이터셋 준비

In [ ]:
train2 = train.copy()

In [ ]:
## fiberID는 어떻게 처리하지........?
## 모르면 삭제............

In [ ]:
train2 = train2.drop(['fiberID'], axis=1)

In [ ]:
test2 = test.copy()
test2 = test.drop(['fiberID'], axis=1)

In [ ]:
train2.head()

In [ ]:
train2.shape
test2.shape

In [ ]:
# TRAIN 데이터의 TYPE을 SAMPLE_SUBMISSION에 대응하는 가변수 형태로 변환시킵니다.
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train2['type_num'] = train2['type'].apply(lambda x: to_number(x, column_number))

In [ ]:
train_x = train2.drop(columns=['type', 'type_num'], axis=1)
train_y = train2['type_num']
test_x = test2

In [ ]:
RANDOM_SEED = 0

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, \
                                                  test_size=0.2, random_state=RANDOM_SEED, stratify = train_y)

In [ ]:
## ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesClassifier

# 학습하기
model = ExtraTreesClassifier()
model.fit(X_train, y_train)
# aaa = model.score(X_test, y_test)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

# 평가하기
y_pred = model.predict_proba(test_x)

In [ ]:
y_pred

In [ ]:
submission = pd.DataFrame(data=y_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('submission3.csv', index=True)

In [ ]:
submission.head()

In [ ]:
# 모델 저장하기
import pickle
pickle.dump(model, open('C:/Users/jieun/jieunnnnn/model/model.pkl','wb'))

In [ ]:
# 모델 가져오기
with open('C:/Users/jieun/jieunnnnn/model/model.pkl', 'rb') as file:
    models = pickle.load(file)

## 변수 중요도 살펴보기
각 변수가 트리를 만드는 결정에 얼마나 중요한지를 평가

0과 1 사이의 숫자로, 각 변수에 대해 0은 전혀 사용되지 않았다는 뜻이고, 1은 완벽하게 타깃을 예측했다는 뜻

In [ ]:
model.feature_importances_

In [ ]:
import numpy as np

importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

In [ ]:
import matplotlib.pyplot as plt

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.barh(X.columns, std) 
plt.xlabel('Feature Labels') 
plt.ylabel('Feature Importances') 
plt.title('Comparison of different Feature Importances') 
plt.show() 

In [ ]:
train2.info()

## 최고 성능의 알고리즘 찾기

In [ ]:
from sklearn.model_selection import train_test_split

XX_train, XX_test, yy_train, yy_test = train_test_split(X_scaled, y, 
                                                    test_size=0.2) 

XX_train.shape, XX_test.shape, yy_train.shape, yy_test.shape

In [ ]:
from sklearn.utils.testing import all_estimators
from sklearn.metrics import accuracy_score

# classifier 알고리즘 모두 추출하기--- (*1)
allAlgorithms = all_estimators(type_filter="classifier")

print(allAlgorithms)    
print(len(allAlgorithms))   # 31
print(type(allAlgorithms))  # list

In [ ]:
for(name, algorithm) in allAlgorithms:
    # 각 알고리즘 객체 생성하기 --- (*2)
    clf = algorithm()

    # 학습하고 평가하기 --- (*3)
    clf.fit(XX_train, yy_train)
    yy_pred = clf.predict(XX_test)
    print(name,"의 정답률 = " , accuracy_score(yy_test, yy_pred))

In [ ]:
from sklearn.model_selection import train_test_split

XX_train, XX_test, yy_train, yy_test = train_test_split(X, y, 
                                                    test_size=0.2) 

XX_train.shape, XX_test.shape, yy_train.shape, yy_test.shape

In [ ]:
from sklearn.utils.testing import all_estimators
from sklearn.metrics import accuracy_score

# classifier 알고리즘 모두 추출하기--- (*1)
allAlgorithms = all_estimators(type_filter="classifier")

print(allAlgorithms)    
print(len(allAlgorithms))   # 31
print(type(allAlgorithms))  # list

In [ ]:
for(name, algorithm) in allAlgorithms:
    # 각 알고리즘 객체 생성하기 --- (*2)
    clf = algorithm()

    # 학습하고 평가하기 --- (*3)
    clf.fit(XX_train, yy_train)
    yy_pred = clf.predict(XX_test)
    print(name,"의 정답률 = " , accuracy_score(yy_test, yy_pred))